### Try PICOVOICE's LEOPARD API for generating transcript of audio

**Note:** This has limited uses if you don't pay. 

In [1]:
! pip install "pvleopard>=1.1"

In [1]:
import pvleopard

In [23]:
access_key = 'WrPIFvgrehYkFxNhizR0bM2lifz9mf/sbXGAX0cZPxsNjMRk4KZt1w=='

In [24]:
leopard = pvleopard.create(access_key=access_key)

### Get audio from YouTube video using PyTube

In [25]:
from pytube import YouTube
import os

In [54]:
youtube_url = r'https://www.youtube.com/watch?v=SZjZN7Rs3a8'

audio_filename = 'green_eggs_and_ham_audio.mp4'

youtube = YouTube(youtube_url)

audio_stream = youtube \
    .streams \
    .filter(only_audio=True, audio_codec='opus') \
    .order_by('bitrate') \
    .last()

audio_stream.download(filename=audio_filename)

'C:\\Users\\Administrator\\Desktop\\repos\\auto_caption\\green_eggs_and_ham_audio.mp4'

### Process the audio from the YouTube video using leopard

In [55]:
# process the audio file using leopard:
transcript, words = leopard.process_file(audio_filename)

In [56]:
# "transcript" is a str of the whole transcript
print(transcript[:100])

You're listening to a toadstool in fairness production of green eggs and ham by Dr Seuss be sure to 


In [57]:
# look at 'words' output:
print(f'{type(words) = }')
print(f'{type(words[0]) = }')
print(f'len "words" list: {len(words)}')
print()

print('First 3 word examples:')
for i, leopard_word in enumerate(words[:3]):
    print('Word ', i)
    print(leopard_word)
    print()

type(words) = <class 'list'>
type(words[0]) = <class 'pvleopard._leopard.Word'>
len "words" list: 821

First 3 word examples:
Word  0
Word(word="you're", start_sec=3.5199999809265137, end_sec=3.6480000019073486, confidence=0.6567937135696411)

Word  1
Word(word='listening', start_sec=3.7119998931884766, end_sec=4.064000129699707, confidence=0.9398349523544312)

Word  2
Word(word='to', start_sec=4.159999847412109, end_sec=4.223999977111816, confidence=0.9839518666267395)



In [58]:
# idx of the 4 elements of a leopard.word tuple 
word = 0
start_sec = 1
end_sec = 2
confidence = 3

In [59]:
# look at one word
word0 = words[0]

print('word:', word0[word])
print('start:', word0[start_sec])
print('end:', word0[end_sec])
print('confidence:', word0[confidence])

word: you're
start: 3.5199999809265137
end: 3.6480000019073486
confidence: 0.6567937135696411


## Convert transcript to SRT Format

**SRT = SubRip subtitle** is a format for storing subtitles. 

Example:
```
0
00:00:08,576 --> 00:00:11,711
it's important for you to know how to mix your own colors to make your color

1
00:00:11,840 --> 00:00:16,351
palettes appear cohesive it is also cheaper than buying a tube of paint for every color

...
```
Follow this link [here](https://picovoice.ai/blog/how-to-create-subtitles-for-any-video-with-python/) to read it in detail.

#### Logic for how to break a transcript into sections:
1. If there is a certain duration of silence between two words we consider it an endpoint (i.e. breakpoint). The user is most likely done talking and they or someone else will start a new sentence later.

2. If there is a certain number of words already in the section we should contain it. Otherwise, it makes the screen too crowded.

*The code below from their blog implements this logic:*

In [60]:
from sequence import Sequence

In [61]:
def second_to_timecode(x: float) -> str:
    hour, x = divmod(x, 3600)  # calculate hours
    minute, x = divmod(x, 60)  # calculate minutes
    second, x = divmod(x, 1)  # calculate seconds
    millisecond = int(x * 1000.)  # calculate milliseconds

    # format the timecode in HH:MM:SS,mmm format
    return '%.2d:%.2d:%.2d,%.3d' % (hour, minute, second, millisecond)


# Function to convert a list of words into SRT format with specified parameters
def to_srt(
        words: Sequence(pvleopard.Leopard.Word),
        endpoint_sec = 1.,
        length_limit = 16) -> str:
    
    # Helper function to create a section and add it to the result
    def _helper(end: int) -> None:
        lines.append("%d" % section)  # add ection number
        lines.append(
            "%s --> %s" %  # add time range in SRT format
            (
                second_to_timecode(words[start].start_sec),
                second_to_timecode(words[end].end_sec)
            )
        )
        lines.append(' '.join(x.word for x in words[start:(end + 1)]))  # combine words in section
        lines.append('')  #add an empty line to separate sections

    lines = list()  # initialize a list to store SRT lines
    section = 0  # initialize section number
    start = 0  # initialize start idx of the current section
    
    # iterate through the words to split them into sections
    for k in range(1, len(words)):
        # check for the amount of silence between words to see if the length limit is reached
        if ((words[k].start_sec - words[k - 1].end_sec) >= endpoint_sec) or \
                (length_limit is not None and (k - start) >= length_limit):
            _helper(k - 1)  # create a section and add it to the result
            start = k  # update start idx
            section += 1  # increment the section number
            
    _helper(len(words) - 1)  # create the final section
    
    # return the SRT-formatted transcript by joining the lines with line breaks
    return '\n'.join(lines)


def save_subtitles(
        words: Sequence(pvleopard.Leopard.Word),
        subtitle_path):
    
    with open(subtitle_path, 'w') as srt_file:
        srt_file.write(to_srt(words))

In [62]:
subtitle_filename = 'subtitles_for_green_eggs_and_ham.srt'

save_subtitles(words, subtitle_filename)

## Add subtitles to the video using moviepy

In [4]:
# configure the environment variable as follows
os.environ["IMAGEMAGICK_BINARY"] = r"C:\Program Files\ImageMagick-7.1.1-Q16-HDRI\magick.exe"

# confirm that it has worked
os.getenv("IMAGEMAGICK_BINARY")

'C:\\Program Files\\ImageMagick-7.1.1-Q16-HDRI\\magick.exe'

In [6]:
# Specify the full path to ImageMagick binary
magick_binary = r'C:\Program Files\ImageMagick-7.1.1-Q16-HDRI\magick.exe'

# Set the ImageMagick binary path for MoviePy
from moviepy.config import change_settings
change_settings({"IMAGEMAGICK_BINARY": magick_binary})

In [37]:
# Run this line of code to add ImageMagick to your PATH
# Confirm that the filepath to the ImageMagick binary file is correct
! PATH=%PATH%;C:\Program Files\ImageMagick-7.1.1-Q16-HDRI

# run this line to confirm that ImageMagick is indeed in your PATH
! magick --version

Version: ImageMagick 7.1.1-21 Q16-HDRI x64 58d86e4:20231021 https://imagemagick.org
Copyright: (C) 1999 ImageMagick Studio LLC
License: https://imagemagick.org/script/license.php
Features: Channel-masks(64-bit) Cipher DPC HDRI Modules OpenCL OpenMP(2.0) 
Delegates (built-in): bzlib cairo flif freetype gslib heic jng jp2 jpeg jxl lcms lqr lzma openexr pangocairo png ps raqm raw rsvg tiff webp xml zip zlib
Compiler: Visual Studio 2022 (193532217)


In [68]:
"""
The only way to customize the style of subtitles is to make a function to 
pass as an argument (make_textclip) to the SubtitlesClip.__init__() function
Therefore, I've defined the function to my desired specifications below.
"""
def make_textclip(txt):
    return TextClip(
        txt,
        font="Georgia-Bold",
        fontsize = 30,  ##TODO - Change this to be based on the file-size
        color="blue",
        stroke_color="black",
        stroke_width=1,
    )

In [69]:
from moviepy.editor import VideoFileClip, CompositeVideoClip
from moviepy.video.tools.subtitles import SubtitlesClip
from moviepy.video.VideoClip import TextClip

# import video stream from youtube object
video_stream = youtube \
    .streams \
    .filter(file_extension='mp4', progressive=True) \
    .get_highest_resolution()

# download video without subtitles
video_without_subtitles_filename = 'green_eggs_and_ham_without_subtitles.mp4'
video_stream.download(filename=video_without_subtitles_filename)

# load video into moviepy.editor.VideoFileClip object
video = VideoFileClip(video_without_subtitles_filename)

# Load the subtitles
subtitles = SubtitlesClip(subtitle_filename, make_textclip=make_textclip)

# Set the position and duration of the subtitles
subtitles = subtitles \
    .set_position(("center", "bottom")) \
    .set_duration(video.duration)

# Composite the video and subtitles
final_clip = CompositeVideoClip([video, subtitles])

# Write the final clip to a video file
video_filename = 'green_eggs_and_ham_with_subtitles.mp4'
final_clip.write_videofile(video_filename, fps=24, codec="libx264")

                                                                                                                       
t:   0%|                                                                  | 2/1368 [6:20:20<02:42,  8.42it/s, now=None]
                                                                                                                       
t:  46%|████████████████████████████▌                                 | 4753/10338 [6:06:04<03:45, 24.72it/s, now=None]

Moviepy - Building video green_eggs_and_ham_with_subtitles.mp4.
MoviePy - Writing audio in green_eggs_and_ham_with_subtitlesTEMP_MPY_wvf_snd.mp3




chunk:   0%|                                                                        | 0/9498 [00:00<?, ?it/s, now=None]

chunk:   0%|                                                                | 2/9498 [00:00<09:37, 16.43it/s, now=None]

chunk:   1%|▎                                                             | 48/9498 [00:00<00:37, 251.98it/s, now=None]

chunk:   1%|▊                                                            | 126/9498 [00:00<00:19, 468.69it/s, now=None]

chunk:   2%|█▎                                                           | 199/9498 [00:00<00:16, 552.54it/s, now=None]

chunk:   3%|█▋                                                           | 265/9498 [00:00<00:15, 578.21it/s, now=None]

chunk:   4%|██▏                                                          | 336/9498 [00:00<00:15, 608.69it/s, now=None]

chunk:   4%|██▌                                                          | 398/9498 [00:00<00:15, 589.79it/s, now=None]

chunk:   5%|██▉               

chunk:  35%|████████████████████▊                                       | 3289/9498 [00:07<00:13, 463.55it/s, now=None]

chunk:  35%|█████████████████████                                       | 3336/9498 [00:07<00:13, 455.97it/s, now=None]

chunk:  36%|█████████████████████▍                                      | 3385/9498 [00:07<00:13, 448.19it/s, now=None]

chunk:  36%|█████████████████████▋                                      | 3433/9498 [00:07<00:13, 449.27it/s, now=None]

chunk:  37%|██████████████████████                                      | 3488/9498 [00:07<00:12, 468.11it/s, now=None]

chunk:  37%|██████████████████████▎                                     | 3537/9498 [00:07<00:12, 470.57it/s, now=None]

chunk:  38%|██████████████████████▋                                     | 3589/9498 [00:07<00:12, 470.22it/s, now=None]

chunk:  38%|███████████████████████                                     | 3641/9498 [00:08<00:12, 466.17it/s, now=None]

chunk:  39%|████████████████████

chunk:  65%|███████████████████████████████████████▎                    | 6218/9498 [00:14<00:07, 424.35it/s, now=None]

chunk:  66%|███████████████████████████████████████▌                    | 6263/9498 [00:14<00:07, 414.36it/s, now=None]

chunk:  66%|███████████████████████████████████████▊                    | 6305/9498 [00:14<00:08, 396.94it/s, now=None]

chunk:  67%|████████████████████████████████████████                    | 6347/9498 [00:14<00:08, 389.29it/s, now=None]

chunk:  67%|████████████████████████████████████████▎                   | 6387/9498 [00:14<00:08, 384.34it/s, now=None]

chunk:  68%|████████████████████████████████████████▌                   | 6428/9498 [00:14<00:07, 388.59it/s, now=None]

chunk:  68%|████████████████████████████████████████▉                   | 6474/9498 [00:14<00:07, 407.33it/s, now=None]

chunk:  69%|█████████████████████████████████████████▏                  | 6515/9498 [00:15<00:07, 394.97it/s, now=None]

chunk:  69%|████████████████████

...

t:  94%|███████████████████████████████████████████████████████████▉    | 9685/10338 [07:00<00:26, 24.41it/s, now=None]

t:  94%|███████████████████████████████████████████████████████████▉    | 9688/10338 [07:00<00:27, 23.45it/s, now=None]

t:  94%|███████████████████████████████████████████████████████████▉    | 9691/10338 [07:00<00:26, 24.36it/s, now=None]

t:  94%|████████████████████████████████████████████████████████████    | 9694/10338 [07:00<00:25, 24.93it/s, now=None]

t:  94%|████████████████████████████████████████████████████████████    | 9697/10338 [07:00<00:26, 23.92it/s, now=None]

t:  94%|████████████████████████████████████████████████████████████    | 9700/10338 [07:00<00:27, 23.53it/s, now=None]

t:  94%|████████████████████████████████████████████████████████████    | 9703/10338 [07:00<00:27, 23.40it/s, now=None]

t:  94%|████████████████████████████████████████████████████████████    | 9706/10338 [07:00<00:26, 24.14it/s, now=None]

t:  94%|████████████████████████

t:  96%|█████████████████████████████████████████████████████████████▏  | 9886/10338 [07:09<00:18, 24.47it/s, now=None]

t:  96%|█████████████████████████████████████████████████████████████▏  | 9889/10338 [07:09<00:17, 25.11it/s, now=None]

t:  96%|█████████████████████████████████████████████████████████████▏  | 9892/10338 [07:09<00:17, 25.80it/s, now=None]

t:  96%|█████████████████████████████████████████████████████████████▎  | 9895/10338 [07:09<00:17, 25.01it/s, now=None]

t:  96%|█████████████████████████████████████████████████████████████▎  | 9898/10338 [07:09<00:17, 25.49it/s, now=None]

t:  96%|█████████████████████████████████████████████████████████████▎  | 9901/10338 [07:09<00:17, 24.71it/s, now=None]

t:  96%|█████████████████████████████████████████████████████████████▎  | 9904/10338 [07:09<00:18, 23.98it/s, now=None]

t:  96%|█████████████████████████████████████████████████████████████▎  | 9908/10338 [07:10<00:16, 25.80it/s, now=None]

t:  96%|████████████████████████

t:  98%|█████████████████████████████████████████████████████████████▋ | 10119/10338 [07:17<00:07, 29.21it/s, now=None]

t:  98%|█████████████████████████████████████████████████████████████▋ | 10123/10338 [07:17<00:07, 29.61it/s, now=None]

t:  98%|█████████████████████████████████████████████████████████████▋ | 10126/10338 [07:17<00:07, 29.28it/s, now=None]

t:  98%|█████████████████████████████████████████████████████████████▋ | 10130/10338 [07:17<00:06, 30.33it/s, now=None]

t:  98%|█████████████████████████████████████████████████████████████▊ | 10134/10338 [07:17<00:06, 31.03it/s, now=None]

t:  98%|█████████████████████████████████████████████████████████████▊ | 10138/10338 [07:17<00:06, 31.87it/s, now=None]

t:  98%|█████████████████████████████████████████████████████████████▊ | 10142/10338 [07:18<00:06, 31.77it/s, now=None]

t:  98%|█████████████████████████████████████████████████████████████▊ | 10146/10338 [07:18<00:05, 33.02it/s, now=None]

t:  98%|████████████████████████

t: 100%|██████████████████████████████████████████████████████████████▉| 10323/10338 [07:25<00:00, 21.61it/s, now=None]

t: 100%|██████████████████████████████████████████████████████████████▉| 10326/10338 [07:25<00:00, 22.46it/s, now=None]

t: 100%|██████████████████████████████████████████████████████████████▉| 10329/10338 [07:25<00:00, 23.94it/s, now=None]

t: 100%|██████████████████████████████████████████████████████████████▉| 10332/10338 [07:25<00:00, 24.04it/s, now=None]

t: 100%|██████████████████████████████████████████████████████████████▉| 10335/10338 [07:25<00:00, 24.70it/s, now=None]

t: 100%|███████████████████████████████████████████████████████████████| 10338/10338 [07:25<00:00, 25.48it/s, now=None]

                                                                                                                       
t:   0%|                                                                  | 2/1368 [6:28:13<02:42,  8.42it/s, now=None]
                                  

Moviepy - Done !
Moviepy - video ready green_eggs_and_ham_with_subtitles.mp4


In [ ]:
from moviepy.editor import VideoFileClip
from IPython.display import Video
# Load the video file
video = VideoFileClip(video_filename)

# Display the video in the Jupyter Notebook
display(Video(video_filename, width=240))

## Addressing issue of the text going off the screen using text-wrapping

In [48]:
import re

In [52]:
video = VideoFileClip(video_without_subtitles_filename)

# Read the contents of the .srt file
subtitles_filename = 'subtitles_to_test_leopard.srt'
with open(subtitles_filename, 'r', encoding='utf-8') as file:
    subtitles_text = file.read()

# Extract the subtitle text (assuming .srt format)
subtitles_text = re.sub(r'\d+\n', '', subtitles_text)  # Remove line numbers
subtitles_text = re.sub(r'\n', ' ', subtitles_text)  # Replace line breaks with spaces

# font specifications
font_size = 30
text_color = 'white'

subtitles = TextClip(subtitles_text, fontsize=font_size, color=text_color)

# Get the video width and height
video_width, video_height = video.size

# Define the left and right margins for the subtitles
left_margin = 20  # Adjust as needed
right_margin = 20  # Adjust as needed

# Calculate the maximum width for subtitles
max_width = video_width - left_margin - right_margin

# Split the subtitle text into lines that fit within the maximum width
subtitles = subtitles.set_position(('left', 'bottom')).set_duration(video.duration)

# Calculate the duration of the subtitle clip based on the text length
subtitle_duration = subtitles.duration
subtitle_duration_per_char = subtitle_duration / len(subtitles_text)

# Create a TextClip for each line
lines = []
current_text = ''
current_duration = 0
for char in subtitles_text:
    current_text += char
    current_duration += subtitle_duration_per_char
    if current_duration >= subtitle_duration or len(current_text) == len(subtitles_text):
        line = TextClip(current_text, fontsize=font_size, color=text_color)
        line = line.set_position(('left', 'bottom')).set_duration(current_duration)
        lines.append(line)
        current_text = ''
        current_duration = 0

# Overlay the wrapped subtitles on the video
subtitles = CompositeVideoClip(lines)
final_clip = CompositeVideoClip([video.set_duration(video.duration), subtitles])

# Write the final clip to a video file
final_clip.write_videofile(f'{video_filename[:-4]}_with_text-wrapping.mp4', fps=24, codec="libx264")

t:   0%|                                                                    | 2/1368 [00:34<02:42,  8.42it/s, now=None]

Moviepy - Building video blacktail_video_with_subtitles_with_text-wrapping.mp4.
MoviePy - Writing audio in blacktail_video_with_subtitles_with_text-wrappingTEMP_MPY_wvf_snd.mp3



t:   0%|                                                                    | 2/1368 [00:37<02:42,  8.42it/s, now=None]

MoviePy - Done.
Moviepy - Writing video blacktail_video_with_subtitles_with_text-wrapping.mp4




t:   5%|███▏                                                               | 66/1368 [00:17<05:28,  3.96it/s, now=None]


...

t:  87%|████████████████████████████████████████████████████████▍        | 1188/1368 [05:35<00:44,  4.09it/s, now=None]


t:  92%|███████████████████████████████████████████████████████████▌     | 1254/1368 [05:52<00:27,  4.18it/s, now=None]


t:  96%|██████████████████████████████████████████████████████████████▋  | 1320/1368 [06:09<00:12,  3.99it/s, now=None]


t:   0%|                                                                    | 2/1368 [07:02<02:42,  8.42it/s, now=None]

Moviepy - Done !
Moviepy - video ready blacktail_video_with_subtitles_with_text-wrapping.mp4
